#  <center>PROJET TEXT MINING </center>

# Importation des bibliothèques

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.probability import FreqDist
import pandas as pd
import re
import tweepy
from IPython.display import display
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')

# Scraping data depuis twitter

In [ ]:
import tweepy
import pandas as pd

#  Identifiants de l’API Twitter
consumer_key = "______"
consumer_secret = "______"
access_token = "______"
access_token_secret = "______"

# Authentifier avec l’API Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Définir les paramètres de la requête
query = "crypto"
count =1000

# Rechercher les tweets contenant le mot-clé
tweets = tweepy.Cursor(api.search_tweets, q=query, lang='en').items(count)

# Convertir les tweets en pandas DataFrame
df = pd.DataFrame([tweet._json for tweet in tweets],columns=["id", "created_at", "text", "retweet_count", "favorite_count"])

In [ ]:
df.to_csv('twitter_crypto_data.csv', index=False)

In [5]:
df_original = pd.read_csv('twitter_crypto_data.csv')

In [6]:
df = df_original.copy()
df = df.drop(columns=['created_at','retweet_count','favorite_count'])

In [7]:
pd.set_option('display.max_colwidth', None) 

df

,id,text
0,1655551533776289794,RT @limewire: 🚨 The LimeWire Token Public Sale is LIVE!\n\nYou can now purchase #LMWR on our launchpad (crypto) and on Republic (fiat &amp; credi…
1,1655551533424033795,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…
2,1655551533029765120,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…
3,1655551532761227264,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…
4,1655551532048293889,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…
...,...,...
995,1655551326279876611,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…
996,1655551326129106944,"RT @rovercrc: You receive $10,000, and you can only buy one of the following #Crypto. What do you choose? 👇 \n\n- $BTC \n- $ETH\n- $SOL\n- $XR…"
997,1655551325382352896,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…
998,1655551325181034496,RT @sync_swap: We are launching #zkSync Airdrop redistribution for our community.\n\nCheck eligibility and claim $zkSync on our site: https:/…


# Data cleaning and normalisation

In [8]:
# Supprimer les caractères spéciaux et les liens de la colonne 'text'
pattern = r'(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?'

df['text'] = df['text'].apply(lambda x: re.sub(pattern, '', x))

df['text'] = df['text'].str.replace(r'^RT[\s:]+', '', regex=True)

In [9]:
# Gérer les valeurs manquantes 
df.dropna(subset=['text'], inplace=True)

In [10]:
# Gérer les valeurs dupliqués 
df.drop_duplicates(subset=['text'], inplace=True)

In [11]:
# Convertir le texte en minuscules
df['text'] = df['text'].str.lower()

# Supprimer la ponctuation et les espaces blancs supplémentaires
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))

In [12]:
df

,id,text
0,1655551533776289794,limewire the limewire token public sale is liveyou can now purchase lmwr on our launchpad crypto and on republic fiat amp credi
1,1655551533424033795,syncswap we are launching zksync airdrop redistribution for our communitycheck eligibility and claim zksync on our site
13,1655551531226103810,solokaybear gmgm
14,1655551531050217473,mcdtoken the mcd team is giving away 3 eth total and 10 million mcd tokens each to three lucky winnersall you have to do is sim
19,1655551530706128898,sharbitoken we love a straight shooting king our sharbi queen only wants the very best thank you to the real king cryptobitlord7
...,...,...
976,1655551329803321345,taskoncampaigns dont miss taskonxyz x dreampadorg special giveaway 500000 dpad for lucky winners may 4th 1000 ma
985,1655551328435982338,bitmartfutures maximize your rewards with bitmartfutures join our daily checkin share your crypto insights amp earn usdt re
991,1655551326510522371,shivst3r gmgm
992,1655551326464405506,bridgeoracle uniswap arbitrum arbiscan cryptoinsightsx crypto sinaestavi thanks for bringing brg to


# Stopwords, Tokenisation et Stemming

In [13]:
# Tokenize la colonne de texte
df['tokens'] = df['text'].apply(word_tokenize)

# Supprimer les stopwords
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# Initialiser le PorterStemmer
stemmer = PorterStemmer()

# Appliquer le stemming à la colonne 'tokens'
df['tokens'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])


In [14]:
df.head()

,id,text,tokens
0,1655551533776289794,limewire the limewire token public sale is liveyou can now purchase lmwr on our launchpad crypto and on republic fiat amp credi,"[limewir, limewir, token, public, sale, livey, purchas, lmwr, launchpad, crypto, republ, fiat, amp, credi]"
1,1655551533424033795,syncswap we are launching zksync airdrop redistribution for our communitycheck eligibility and claim zksync on our site,"[syncswap, launch, zksync, airdrop, redistribut, communitycheck, elig, claim, zksync, site]"
13,1655551531226103810,solokaybear gmgm,"[solokaybear, gmgm]"
14,1655551531050217473,mcdtoken the mcd team is giving away 3 eth total and 10 million mcd tokens each to three lucky winnersall you have to do is sim,"[mcdtoken, mcd, team, give, away, 3, eth, total, 10, million, mcd, token, three, lucki, winnersal, sim]"
19,1655551530706128898,sharbitoken we love a straight shooting king our sharbi queen only wants the very best thank you to the real king cryptobitlord7,"[sharbitoken, love, straight, shoot, king, sharbi, queen, want, best, thank, real, king, cryptobitlord7]"


# Les termes les plus courants

In [15]:
# Concaténer tous les tokens dans une seule liste
all_tokens = [token for tokens in df['tokens'] for token in tokens]

# Calculer la fréquence des échéances
freq_dist = FreqDist(all_tokens)

# Afficher les termes les plus courants
most_common = freq_dist.most_common(15)

In [16]:
word_freq = pd.DataFrame(most_common, columns=['word', 'frequency'])

In [17]:
word_freq

,word,frequency
0,crypto,49
1,follow,25
2,amp,23
3,airdrop,14
4,rt,14
5,token,13
6,giveaway,13
7,like,13
8,new,11
9,x,10


# Sentiment Analysis

In [22]:
# Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    text = row['text']

    # Effectuer une analyse des sentiments
    sentiment = sia.polarity_scores(text)
    sentiment_score = sentiment['compound']  # compound score ranges from -1 to 1

    # Assign sentiment labels based on the score
    if sentiment_score >= 0.05:
        sentiment_label = 'positive'
    elif sentiment_score <= -0.05:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'

    df.loc[index, 'sentiment_score'] = sentiment_score
    df.loc[index, 'sentiment_label'] = sentiment_label


In [23]:
df

,id,text,tokens,sentiment_score,sentiment_label
0,1655551533776289794,limewire the limewire token public sale is liveyou can now purchase lmwr on our launchpad crypto and on republic fiat amp credi,"[limewir, limewir, token, public, sale, livey, purchas, lmwr, launchpad, crypto, republ, fiat, amp, credi]",0.0000,neutral
1,1655551533424033795,syncswap we are launching zksync airdrop redistribution for our communitycheck eligibility and claim zksync on our site,"[syncswap, launch, zksync, airdrop, redistribut, communitycheck, elig, claim, zksync, site]",0.0000,neutral
13,1655551531226103810,solokaybear gmgm,"[solokaybear, gmgm]",0.0000,neutral
14,1655551531050217473,mcdtoken the mcd team is giving away 3 eth total and 10 million mcd tokens each to three lucky winnersall you have to do is sim,"[mcdtoken, mcd, team, give, away, 3, eth, total, 10, million, mcd, token, three, lucki, winnersal, sim]",0.6369,positive
19,1655551530706128898,sharbitoken we love a straight shooting king our sharbi queen only wants the very best thank you to the real king cryptobitlord7,"[sharbitoken, love, straight, shoot, king, sharbi, queen, want, best, thank, real, king, cryptobitlord7]",0.9242,positive
...,...,...,...,...,...
976,1655551329803321345,taskoncampaigns dont miss taskonxyz x dreampadorg special giveaway 500000 dpad for lucky winners may 4th 1000 ma,"[taskoncampaign, dont, miss, taskonxyz, x, dreampadorg, special, giveaway, 500000, dpad, lucki, winner, may, 4th, 1000]",0.8420,positive
985,1655551328435982338,bitmartfutures maximize your rewards with bitmartfutures join our daily checkin share your crypto insights amp earn usdt re,"[bitmartfutur, maxim, reward, bitmartfutur, join, daili, checkin, share, crypto, insight, amp, earn, usdt]",0.7579,positive
991,1655551326510522371,shivst3r gmgm,"[shivst3r, gmgm]",0.0000,neutral
992,1655551326464405506,bridgeoracle uniswap arbitrum arbiscan cryptoinsightsx crypto sinaestavi thanks for bringing brg to,"[bridgeoracl, uniswap, arbitrum, arbiscan, cryptoinsightsx, crypto, sinaestavi, thank, bring, brg]",0.4404,positive


# Les tweets neutres

In [26]:
neutral_rows = df[['text','sentiment_score', 'sentiment_label']][df['sentiment_label'] == 'neutral']

In [27]:
neutral_rows

,text,sentiment_score,sentiment_label
0,limewire the limewire token public sale is liveyou can now purchase lmwr on our launchpad crypto and on republic fiat amp credi,0.0,neutral
1,syncswap we are launching zksync airdrop redistribution for our communitycheck eligibility and claim zksync on our site,0.0,neutral
13,solokaybear gmgm,0.0,neutral
21,stronghedge nb bro as far as big breakfast go looks above average quality,0.0,neutral
22,christran147 dcaceeeeewill giveaway some snek if this tweet get enough engagementlike and retweetfollowtag 3 snek or potent,0.0,neutral
...,...,...,...
938,missycrypto done,0.0,neutral
963,chitaglorya3rd 150 2250000 idr in 48 hours 50rt follow bitindichain 100rt quote tweet amp comment with bni,0.0,neutral
970,current doge price is 00749588 dogecoin crypto tothemoon,0.0,neutral
991,shivst3r gmgm,0.0,neutral


# Les tweets Positives 

In [28]:
positive_rows = df[['text','sentiment_score', 'sentiment_label']][df['sentiment_label'] == 'positive']

In [29]:
positive_rows

,text,sentiment_score,sentiment_label
14,mcdtoken the mcd team is giving away 3 eth total and 10 million mcd tokens each to three lucky winnersall you have to do is sim,0.6369,positive
19,sharbitoken we love a straight shooting king our sharbi queen only wants the very best thank you to the real king cryptobitlord7,0.9242,positive
23,zklinkorg 1 introducing zklinks 72hr dunkirk test a gamechanger for crypto safety well simulate an emergency shutdown gi,0.3182,positive
26,new big fcfs airdrop metastarx follow metastarx like amp rt amp tag 3 friends free 3000 worth of,0.8689,positive
28,mrdiamondhandz1 good morning lunc army im glad we are starting to see the true colors of people now that we have a nice shake out,0.8689,positive
...,...,...,...
909,raysunited real open mainnet date to be confirmed the ladder of success base mining rate increased ray reacts live ep,0.7003,positive
952,cryptotechdao arbraca x cryptotech big airdrop rewards 200 usdt 3 nfts follow arbraca3 like rt and tag 3 friends,0.8271,positive
976,taskoncampaigns dont miss taskonxyz x dreampadorg special giveaway 500000 dpad for lucky winners may 4th 1000 ma,0.8420,positive
985,bitmartfutures maximize your rewards with bitmartfutures join our daily checkin share your crypto insights amp earn usdt re,0.7579,positive


# Les tweets Négatives 

In [30]:
negative_rows = df[['text','sentiment_score', 'sentiment_label']][df['sentiment_label'] == 'negative']

In [35]:
negative_rows

,text,sentiment_score,sentiment_label
57,crypto ftx is seeking to claw back close to 39 billion of cash and crypto from bankrupt digital asset lender genesis and a nonbankr,-0.2732,negative
105,owlofatena cant wait for x1 a trustless bridge between bitcoin and the evm world is going to be huge this is the holy grail of,-0.2500,negative
264,pauly0x bitboycrypto pathetic little wormno one cares about youdirt monkey,-0.2415,negative
328,idleheroestt selling this stat flip ruby csgo knife for 96 buff which is 3690usd via bankcrypto dm me or comment and ill dm you,-0.4215,negative
355,big7seven cryptobitlord7 mongcoin in reality it is a pump and dump shitcoin with a lot of insider trading,-0.3818,negative
416,maverick23nft i hate crypto more every day,-0.5719,negative
473,tenetorg eva brings unparalleled access to tenetthe firstever default wallet of a chain with no key management private key sh,-0.2960,negative
562,ctjlewis this shit will tell you the bible and the bitcoin whitepaper are gptgenerated and princeton had like no issue with this appa,-0.5106,negative
645,itzuwem gm new weeklets connect and growday 7 no account should have less than 5000 followers follow me first for,-0.2960,negative
672,people open funko pop stores collect millions from preorders ship nothing close shop no one should be worried about crypto,-0.0803,negative


In [36]:
print(len(negative_rows))

17
